In [1]:
import pandas as pd
import numpy as np

In [2]:
data= pd.read_csv('adjprice.csv')

In [3]:
data

,Date,0111145D US Equity,0202445Q US Equity,0203524D US Equity,0226226D US Equity,0376152D US Equity,0440296D US Equity,0544749D US Equity,0574018D US Equity,0598884D US Equity,...,YNR US Equity,YRCW US Equity,YUM US Equity,YUMC US Equity,ZBH US Equity,ZETHQ US Equity,ZION US Equity,ZRN US Equity,ZTS US Equity,ZTS-W US Equity
0,19930907,13.2719,13.6829,8.4429,8.1042,11.000,57.3245,17.8887,6.8315,28.1246,...,NaN,144439.5121,NaN,NaN,NaN,6.875,7.1675,30.7970,NaN,NaN
1,19930908,13.3263,13.5315,8.2147,7.9590,11.000,57.2096,17.8064,6.8315,27.5051,...,NaN,143691.1208,NaN,NaN,NaN,6.750,6.9766,30.0188,NaN,NaN
2,19930909,13.7070,13.3800,8.7852,8.0627,11.125,59.1625,17.6831,6.8315,27.7529,...,NaN,143691.1208,NaN,NaN,NaN,6.625,6.9554,30.0188,NaN,NaN
3,19930910,13.3807,13.4810,9.4127,8.0368,11.125,59.6220,17.6420,6.8773,27.5051,...,NaN,145187.9033,NaN,NaN,NaN,6.750,7.2099,29.6852,NaN,NaN
4,19930911,13.3807,13.4810,9.4127,8.0368,11.125,59.6220,17.6420,6.8773,27.5051,...,NaN,145187.9033,NaN,NaN,NaN,6.750,7.2099,29.6852,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9454,20190727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.3500,114.02,45.31,134.50,NaN,45.1600,NaN,115.11,NaN
9455,20190728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.3500,114.02,45.31,134.50,NaN,45.1600,NaN,115.11,NaN
9456,20190729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.1800,114.10,45.43,134.53,NaN,44.6500,NaN,115.55,NaN
9457,20190730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.1800,113.24,44.00,136.67,NaN,45.4800,NaN,115.79,NaN


In [4]:
data = data[(data['Date']>20000000)&( data['Date']< 20101231 )].dropna(axis=1) # select 2000-2010 year

In [5]:
data=data.reset_index(drop=True) # reset index

In [6]:
value =list(data.iloc[0])[1:]  # filter the stock which the start price not located within IQR (25%-75%) 
                                # remove too expansive or cheap
q3, q1 = np.percentile(value, [75 ,25]) #  get iqr
name  = data.keys() # get stock name
droplst=[]
for i in range(1, len(name)):
    if data.loc[0, name[i]]<q1 or data.loc[0,name[i]]>q3: # if price outside iqr then drop
        droplst.append(name[i])
data = data.drop( droplst,axis=1)

In [7]:
stock_names = data.columns[1:]

In [8]:
stock_names

Index(['0111145D US Equity', '0203524D US Equity', '0948669D US Equity',
       '0961514D US Equity', '1284849D US Equity', '1541931D US Equity',
       '8394653Q US Equity', '9876566D US Equity', 'ABMD US Equity',
       'ABT US Equity',
       ...
       'WAT US Equity', 'WBA US Equity', 'WFC US Equity', 'WM US Equity',
       'WMB US Equity', 'WOR US Equity', 'WY US Equity', 'X US Equity',
       'XEL US Equity', 'XOM US Equity'],
      dtype='object', length=310)

In [9]:
def variance1(data):
    
    # Number of observations
    n = len(data)
    # Mean of the data
    mean = sum(data) / n
    
   # Square deviations
    deviations = [(x - mean) ** 2 for x in data]
   # Variance
    variance = sum(deviations) / n
    return variance

In [10]:
variance =[]
for i in stock_names:
    temp = []
    col = list(data[i])
    for j in range(1,len(col)):
        temp.append((col[j]-col[j-1])/col[j-1])
    variance.append(variance1(temp))
#     variance

In [11]:
data = data.melt(id_vars = 'Date') # pivot table/convert to recommond format

In [12]:
data

,Date,variable,value
0,20000101,0111145D US Equity,18.6789
1,20000102,0111145D US Equity,18.6789
2,20000103,0111145D US Equity,18.1041
3,20000104,0111145D US Equity,18.2119
4,20000105,0111145D US Equity,18.7148
...,...,...,...
1245265,20101226,XOM US Equity,57.0210
1245266,20101227,XOM US Equity,56.8730
1245267,20101228,XOM US Equity,57.1924
1245268,20101229,XOM US Equity,57.1534


In [13]:
start =pd.DataFrame( data.groupby(['variable']).first()['value']) # get start price for each stock 

In [14]:
end = pd.DataFrame( data.groupby(['variable']).last()['value']) # get end price for each stock 

In [15]:
variacne = pd.DataFrame( data.groupby(['variable']).var()['value']) # get variance for each stock 

In [16]:
# create new data frame to store the value
stockname= list(start.index)
new_data= pd.DataFrame({'stock':stockname, 'start_price': list(start['value']),'end_price':list(end['value']),'variance':list(variacne['value'])})

In [17]:
data

,Date,variable,value
0,20000101,0111145D US Equity,18.6789
1,20000102,0111145D US Equity,18.6789
2,20000103,0111145D US Equity,18.1041
3,20000104,0111145D US Equity,18.2119
4,20000105,0111145D US Equity,18.7148
...,...,...,...
1245265,20101226,XOM US Equity,57.0210
1245266,20101227,XOM US Equity,56.8730
1245267,20101228,XOM US Equity,57.1924
1245268,20101229,XOM US Equity,57.1534


In [91]:
new_data['increasing_rate']= (new_data['end_price']-new_data['start_price'])/new_data['start_price'] # calculating the increasing rate and store

C:\Users\klin0\AppData\Local\Temp\ipykernel_26220\165184746.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['increasing_rate']= (new_data['end_price']-new_data['start_price'])/new_data['start_price'] # calculating the increasing rate and store


In [92]:
new_data

,stock,start_price,end_price,variance,increasing_rate
0,0111145D US Equity,18.6789,48.4923,0.000218,1.596100
2,0948669D US Equity,16.9007,20.6623,0.000762,0.222571
3,0961514D US Equity,15.3570,40.1340,0.000138,1.613401
4,1284849D US Equity,22.6540,68.1464,0.000285,2.008140
5,1541931D US Equity,22.3590,33.5659,0.000910,0.501225
...,...,...,...,...,...
304,WMB US Equity,13.1106,13.9472,0.001254,0.063811
305,WOR US Equity,8.9766,15.7393,0.000581,0.753370
307,X US Equity,25.8487,55.1283,0.000799,1.132730
308,XEL US Equity,8.4109,17.7181,0.000289,1.106564


In [93]:
for i in range(new_data.shape[0]):
    new_data.loc[i,'variance'] = variance[i]

C:\Users\klin0\AppData\Local\Temp\ipykernel_26220\255587603.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.loc[i,'variance'] = variance[i]


In [94]:
new_data = new_data[new_data['increasing_rate']>0]

In [95]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    x= 1/x
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference
def normalize(v):
    norm=np.linalg.norm(v)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm

In [96]:
import tqdm
iters =1000000
max_in = 0
permu = None
weight_best =None
for k in [3,5,7,9,11]:
    for i in tqdm.tqdm(range(iters)):
        per = new_data.sample(n=k).reset_index()
        weight= list(softmax(normalize(per['variance'])))
        sum_increase = sum([per.loc[i,'increasing_rate']*weight[i] for i in range(per.shape[0])])
        if sum_increase > max_in:
            max_in = sum_increase
            permu=per
            weight_best=weight

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1727.30it/s]


In [97]:
permu

,index,stock,start_price,end_price,variance,increasing_rate
0,186,MEE US Equity,10.1424,53.6173,0.001096,4.286451
1,220,PKI US Equity,16.6617,24.4976,0.000571,0.470294
2,53,BLK US Equity,12.3490,153.4430,0.000446,11.425541


In [100]:
import itertools
pp = []
for i in tqdm.tqdm(itertools.permutations([i/100 for i in range(100) ], len(weight_best))):
    if sum(i)==1 and min(i)>0.1 and min(i)<:
        pp.append(i)

720it [00:00, 1340390.09it/s]


In [101]:
max_inc= max_in
new_weight= None
for i in range(len(pp)):
    sum_in = sum([pp[i][j]*permu.loc[j,'increasing_rate'] for j in range(len(pp[i]))])
    if sum_in>max_inc:
        max_inc=sum_in
        new_weight= pp[i]
if not new_weight:
    new_weight = weight

In [103]:
new_weight

(0.1, 0.0, 0.9)

In [104]:
permu

,index,stock,start_price,end_price,variance,increasing_rate
0,186,MEE US Equity,10.1424,53.6173,0.001096,4.286451
1,220,PKI US Equity,16.6617,24.4976,0.000571,0.470294
2,53,BLK US Equity,12.3490,153.4430,0.000446,11.425541


In [107]:
money =10000
static_weight = [money*new_weight[i]//permu.loc[i,'end_price'] for i in range(len(new_weight))]

In [109]:
for i in range(len(new_weight)):
    print('Stock Name: ', permu.loc[i,'stock'],end=' ')
    print('Buying Number: ', static_weight[i])

Stock Name:  MEE US Equity Buying Number:  18.0
Stock Name:  PKI US Equity Buying Number:  0.0
Stock Name:  BLK US Equity Buying Number:  58.0


In [64]:
import pandas as pd
import numpy as np

In [65]:
stock = ['Date','AMG US Equity','CCK US Equity','BLK US Equity','TEN US Equity','REGN US Equity']
weight = [0.02,0,0.94,0.03,0.01]

In [75]:
data =pd.read_csv('adjprice.csv')
data = data[stock]
data = data[(data['Date']>20000000)&( data['Date']< 20101231 )].reset_index()

In [76]:
def variance1(data):
    
    # Number of observations
    n = len(data)
    # Mean of the data
    mean = sum(data) / n
    
   # Square deviations
    deviations = [(x - mean) ** 2 for x in data]
   # Variance
    variance = sum(deviations) / n
    return variance

In [77]:
def overall(data,stock):
    ranges= []
    risk= []
    rate= []
    for i in range(1,len(stock)):
        ranges.append(max(data[stock[i]])-min(data[stock[i]]))
        rate.append((float( data[stock[1]].tail(1))-float(data[stock[1]].head(1)))/float(data[stock[i]].head(1)))
        temp_in =[]
        for j in range(1,data.shape[0]):
            temp_in.append((data.loc[j,stock[i]] - data.loc[j-1,stock[i]])/data.loc[j,stock[i]])
        risk.append(variance1(temp_in))
    # cut dominate 
    return ranges, rate,risk


In [149]:
def dynamic_weight(data,stock,weight):
    whole_ranges=[]
    whole_risk=[]
    whole_rate=[]
    recent_ranges= []
    recent_risk=[]
    recent_rate=[]
    whole_ranges,whole_rate,whole_risk = overall(data,stock)
    recent_ranges,recent_rate,recent_risk = overall(data.tail(100).reset_index(),stock)

#     print( whole_ranges,whole_risk,whole_rate)
#     print(  recent_ranges,recent_risk,recent_rate)
    ranges_change_rate= []
    risk_change_rate=[]
    rate_chage = []
    for i in range(len(whole_ranges)):
        ranges_change_rate.append(( recent_ranges[i]- whole_ranges[i])/whole_ranges[i])
        
        risk_change_rate.append(( recent_risk[i]- whole_risk[i])/whole_risk[i])
        
        rate_chage.append(( recent_rate[i]- whole_rate[i])/whole_rate[i])
    
#     return ranges, rate,risk
    joint = []
    for i in range(len(whole_ranges)):
        joint.append(ranges_change_rate[i]*risk_change_rate[i]*rate_chage[i])
    #modify weigth
    max_idex = weight.index(max(weight))
    if max_idex >0.7:
        weight[max_idex] *= 0.8
    
    res = weight[max_idex]*0.2/(len(stock)-2)
    for i in range(len(weight)):
        if i != max_idex:
            weight[i]+=res
            
    for i in range(len(weight)):
        weight[i]=weight[i]*(1+joint[i])
    return (softmax(normalize(weight)))

In [150]:
dynamic_weight(data,stock,weight)

array([0.23316698, 0.1473853 , 0.21250534, 0.23380845, 0.17313393])

In [139]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference
def normalize(v):
    norm=np.linalg.norm(v)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm

In [52]:
softmax(normalize(dynamic_weight(data,stock,weight)[0]))

array([0.00291281, 0.92975797, 0.00299201, 0.00946947, 0.05486775])